In [1]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas
import datetime
import locale


%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


In [2]:
df = pd.read_csv('C:/Users/andre/Documents/dashboard/base_ventas.csv') # Leo la base de datos

In [3]:
df.columns

Index(['ID', 'TICKET_KEY', 'PRODUCTO_KEY', 'TIENDA_KEY', 'TIENDA_CVE',
       'TIENDA_CVE1', 'PLAZA_CVE', 'PLAZA_DES', 'TIENDA_DES', 'ENTORNO_DES',
       'UPC_CVE', 'PRODUCTO_DES', 'SUPERGRUPO_ID', 'CATEGORIA_ID',
       'CATEGORIA_DES', 'SUBCATEGORIA_DES', 'SEGMENTO_DES', 'MES_ID',
       'TIEMPO_KEY', 'TIEMPO_ADM_KEY', 'UNIDADES_NETAS_12',
       'UNIDADESDEVUELTAS_NUM_11', 'VENTA_NETA_SIMP_12', 'VENTA_NETA_CIMP_12',
       'DEVOLUCION_AMT_11', 'DEVOLUCION_AMT_CIMP_11', 'DevVtaTerceros_41',
       'DevVtaTerceros_Uds_41', 'VtaTerceros_42', 'VtaTerceros_Uds_42',
       'Serv_Val_98', 'Serv_Uds_98', 'DescPromo_Val_21', 'DescPromo_Uds_21',
       'DevDescPromo_Val_51', 'DevDescPromo_Uds_51', 'PromTerceros_40',
       'PromTerceros_Uds_40', 'DevPromTerceros', 'DevPromTerceros_Uds', 'TIPO',
       'TIPO_VTA_DOMICILIO', 'VT_NUM_CAJA', 'VT_ID_EMPL', 'VT_HRA_MOV',
       'Total_VT_FOL_PRO', 'SUPERGRUPO_DES'],
      dtype='object')

In [4]:
df_f = df.copy() # Hago una copia de primer data frame
df_f = df_f[['SUPERGRUPO_DES','TIEMPO_ADM_KEY','UNIDADES_NETAS_12',
       'UNIDADESDEVUELTAS_NUM_11', 'VENTA_NETA_SIMP_12','DEVOLUCION_AMT_11']] # Selecciono columnas de interes

In [5]:
df_f['TIEMPO_ADM_KEY'] = pd.to_datetime(df_f['TIEMPO_ADM_KEY']) #transformo la columna en formato fecha
df_f.sort_values(by = 'TIEMPO_ADM_KEY',inplace = True) # organizo la columna de fecha
df_f['mes_year'] = df_f['TIEMPO_ADM_KEY'].dt.strftime('%m-%Y') #Creo una nueva columna de mes-año
df_f['Venta'] = df_f['VENTA_NETA_SIMP_12'] + df_f['DEVOLUCION_AMT_11']# Creo una nueva columna la cual es vente y es la suma de ventas mas devoluciones
df_f['Unidades'] = df_f['UNIDADES_NETAS_12'] + df_f['UNIDADESDEVUELTAS_NUM_11'] # al igual que la columna anterior pero en unidades
df_f.drop(['UNIDADES_NETAS_12',
       'UNIDADESDEVUELTAS_NUM_11', 'VENTA_NETA_SIMP_12','DEVOLUCION_AMT_11'], axis=1, inplace=True) #Elimino las columnas que use para crear las nuevas
df_f['mes'] = pd.to_datetime(df_f['TIEMPO_ADM_KEY']).dt.month # Creo una nueva columa con el mes
df_f = df_f[(~(df_f.SUPERGRUPO_DES.isin(['SN'])))] #Selecciono los supergrupos de interes

In [6]:
df_f['Venta_23'] = df_f.apply(lambda row: row['Venta'] if pd.to_datetime(row['TIEMPO_ADM_KEY']).year == 2023 else 0, axis=1) # Creo una columna con solo las ventas del 2023

In [7]:
def sumar_columnas (row,indicador,year_):
    if pd.to_datetime(row['TIEMPO_ADM_KEY']).year == year_:
        return row[indicador]
    else:
        return 0 # Creo una funcion que me permita sumar registros de con cierta validacion

In [8]:
indicador = 'Venta'
year_ = 2022
df_f['Ventas_22'] = df_f.apply(sumar_columnas, args=(indicador ,year_),axis=1) # Creo una columna de ventas con las ventas del 2022

In [9]:
indicador1 = 'Unidades'
year_2 = 2022
year_1 = 2023
df_f['Unds_22'] = df_f.apply(sumar_columnas, args=(indicador1 ,year_2),axis=1) 
df_f['Unds_23'] = df_f.apply(sumar_columnas, args=(indicador1 ,year_1),axis=1)#   Similar a las anteriores creo las columnas de unidades 2022 y 2023

In [10]:
#df_f.drop(columns= ['pvp_22','pvp_23'], inplace = True)

In [11]:
df_f = df_f.fillna(0)
agrupado= df_f.copy()
agrupado = agrupado.groupby(['SUPERGRUPO_DES','mes']).agg({'Venta_23':np.sum,'Ventas_22':np.sum,'Unds_22':np.sum,'Unds_23':np.sum})
agrupado.reset_index( inplace = True)
agrupado # Como la tabla no se encuentra agrupada, la agrupo por las columnas de supergrupo y mes

,SUPERGRUPO_DES,mes,Venta_23,Ventas_22,Unds_22,Unds_23
0,ANTOJO,1,1646074.63,2468173.20,1288.0,634.0
1,ANTOJO,2,4596514.85,2447042.43,1544.0,1830.0
2,ANTOJO,3,4853506.54,2860153.46,1429.0,1779.0
3,ANTOJO,4,6001944.39,5489184.81,2570.0,2018.0
4,ANTOJO,5,8972516.53,5235296.47,2458.0,3314.0
...,...,...,...,...,...,...
67,SED,8,2300634.48,1954807.53,881.0,795.0
68,SED,9,0.00,4440169.47,1838.0,0.0
69,SED,10,0.00,2280430.34,931.0,0.0
70,SED,11,0.00,2766847.81,1102.0,0.0


In [12]:
agrupado = agrupado[(agrupado['mes'] <8)]
agrupado['Crec_ventas'] = (agrupado['Venta_23']/agrupado['Ventas_22'] -1 )*100
agrupado['Crec_unds'] = (agrupado['Unds_23']/agrupado['Unds_22'] -1 )*100
agrupado['Crec_pvp'] = ((agrupado['Venta_23']/agrupado['Unds_23'])/ (agrupado['Ventas_22']/agrupado['Unds_22'])-1)*100
agrupado
# Creo unas metricas de crecimiento de 2023 vs 2022 de ventas y unidades

C:\Users\andre\AppData\Local\Temp\ipykernel_20640\3175324641.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agrupado['Crec_ventas'] = (agrupado['Venta_23']/agrupado['Ventas_22'] -1 )*100
C:\Users\andre\AppData\Local\Temp\ipykernel_20640\3175324641.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agrupado['Crec_unds'] = (agrupado['Unds_23']/agrupado['Unds_22'] -1 )*100
C:\Users\andre\AppData\Local\Temp\ipykernel_20640\3175324641.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

,SUPERGRUPO_DES,mes,Venta_23,Ventas_22,Unds_22,Unds_23,Crec_ventas,Crec_unds,Crec_pvp
0,ANTOJO,1,1646074.63,2468173.20,1288.0,634.0,-33.307977,-50.776398,35.487895
1,ANTOJO,2,4596514.85,2447042.43,1544.0,1830.0,87.839606,18.523316,58.483252
2,ANTOJO,3,4853506.54,2860153.46,1429.0,1779.0,69.693921,24.492652,36.308382
3,ANTOJO,4,6001944.39,5489184.81,2570.0,2018.0,9.341270,-21.478599,39.250280
4,ANTOJO,5,8972516.53,5235296.47,2458.0,3314.0,71.385070,34.825061,27.116627
5,ANTOJO,6,3933019.88,3087249.98,1348.0,1444.0,27.395576,7.121662,18.926064
6,ANTOJO,7,3194850.67,3483051.88,1443.0,1104.0,-8.274388,-23.492723,19.891358
12,D&R,1,407503.06,549775.06,206.0,91.0,-25.878220,-55.825243,67.792161
13,D&R,2,670335.00,592829.52,163.0,179.0,13.073823,9.815951,2.966665
14,D&R,3,749967.81,527461.94,171.0,170.0,42.184251,-0.584795,43.020629


In [13]:
agrupado.drop(columns=['Venta_23','Ventas_22','Unds_22','Unds_23'],inplace=True)
#columns_to_format = ['Crec_ventas','Crec_unds','Crec_pvp']
#agrupado[columns_to_format] = agrupado[columns_to_format].applymap(lambda x: '{:.2%}'.format(x))
agrupado = agrupado.melt(id_vars=['SUPERGRUPO_DES','mes'],var_name='Indicador',value_name='Valor')
agrupado
# como solo me interesan las columnas de variacion elimino las otras qu fueron base para crear la variacion, y dinamizo las columnas crear para una mejor estructura             

C:\Users\andre\AppData\Local\Temp\ipykernel_20640\49344176.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agrupado.drop(columns=['Venta_23','Ventas_22','Unds_22','Unds_23'],inplace=True)


,SUPERGRUPO_DES,mes,Indicador,Valor
0,ANTOJO,1,Crec_ventas,-33.307977
1,ANTOJO,2,Crec_ventas,87.839606
2,ANTOJO,3,Crec_ventas,69.693921
3,ANTOJO,4,Crec_ventas,9.341270
4,ANTOJO,5,Crec_ventas,71.385070
...,...,...,...,...
121,SED,3,Crec_pvp,23.748306
122,SED,4,Crec_pvp,10.275037
123,SED,5,Crec_pvp,21.421951
124,SED,6,Crec_pvp,20.369098


In [14]:
# Make Dataframe pipeline interactive
idf = agrupado.interactive()

Ventas  traves del tiempo

In [15]:
#define panel widgets
mes_slider = pn.widgets.IntSlider(name = 'Mes_slider', start = 1, end = 8, step = 1, value = 8)
mes_slider

IntSlider(end=8, name='Mes_slider', start=1, value=8)

In [16]:
#radiobutton
yaxis_venta = pn.widgets.RadioButtonGroup(
    name = 'Y axis',
    options =df_f['SUPERGRUPO_DES'].unique().tolist(),
    button_type = 'success'
)

yaxis_venta

RadioButtonGroup(button_type='success', name='Y axis', options=['SED', 'ANTOJO', ...], value='SED')

In [17]:
idf 

In [18]:
indicadores = ['Crec_ventas','Crec_unds','Crec_pvp']
ventas_pipeline = (
        idf[
        (idf.mes <= mes_slider)&
        (idf.SUPERGRUPO_DES == yaxis_venta)
        ].sort_values(by = 'mes')
)
ventas_pipeline

# Tendecia Indicador

In [19]:
ventas_pipeline

In [20]:
ventas_plot = ventas_pipeline.hvplot(x='mes',by='Indicador', y ='Valor', line_width = 2, title='Crecimientos vs Aant')
ventas_plot
                                     

# Tabla dincamica Categoria - Indicador

In [26]:
crecimientos_table = ventas_pipeline.copy()
crecimientos_table = crecimientos_table.pivot(index=['SUPERGRUPO_DES','mes'],columns='Indicador',values='Valor').reset_index()
crecimientos_table =crecimientos_table.pipe(pn.widgets.Tabulator, pagination ='remote', page_size=10, sizing_mode='stretch_width')
crecimientos_table

# Grafica corrrelaciona Crecimiento Ventas vs Unidades

In [27]:
cor_table = idf.copy()
cor_table = (
        cor_table[
        (idf.mes == mes_slider)]
    .sort_values(by = 'mes')
)

cor_table = cor_table.pivot(index=['SUPERGRUPO_DES','mes'],columns='Indicador',values='Valor').reset_index()

In [28]:
cor_table 

In [29]:
pvp_vs_unds_scatterplot = cor_table.hvplot(x='Crec_unds',
                                           y='Crec_pvp',
                                           by='SUPERGRUPO_DES',
                                           size = 80, kind='scatter',
                                           alpha=0.7,
                                           legend = False,
                                           height = 500,
                                           width = 500)

pvp_vs_unds_scatterplot                                         

# Bar Char por indicador

In [30]:
indicadores = agrupado['Indicador'].unique().tolist()
yaxis_indicador_source = pn.widgets.RadioButtonGroup(
    name = 'Y axis',
    options = agrupado['Indicador'].unique().tolist(),
    button_type = 'success'
)
indicador_bar_pipeline = (
    idf[
    (idf.mes == mes_slider) &
    (idf.Indicador == yaxis_indicador_source)].
    groupby(['mes','SUPERGRUPO_DES'])['Valor'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by = 'mes')
    .reset_index(drop = True)
)
    

In [31]:
indicador_bar_pipeline

In [32]:
indicador_bar_plot = indicador_bar_pipeline.hvplot(kind = 'bar'
                                                   , x='SUPERGRUPO_DES'
                                                   ,y='Valor',
                                                   title='Indicadores por supergrupo')
indicador_bar_plot

# Creando el template

In [33]:
template = pn.template.FastListTemplate(
    title = 'Crecimientos supergrupos dashboard',
    sidebar = [pn.pane.Markdown("# Los supergrupos son considerados categorias dentro dse la cadena de oxxo, con este dashboard, se busca analizar los principales crecimientos por precio,ventas y unidades"),
               pn.pane.PNG('oxxo.png', sizing_mode='scale_both'),
               pn.pane.Markdown('## Filtros'),
               mes_slider],
    main=[pn.Row(pn.Column(yaxis_venta,ventas_plot.panel(width=600),margin=(0,25)),
                 crecimientos_table.panel(width=600)),
          pn.Row(pn.Column(pvp_vs_unds_scatterplot.panel(width=600), margin=(0,25)),
                 pn.Column(yaxis_indicador_source, indicador_bar_plot.panel(width=600)))],
    accent_base_color="#BB2124",
    header_background = "#F7AD19",
)
template.show()
                           
                           

Launching server at http://localhost:63742
